In [1]:
import os
import random
import numpy as np
from tensorflow.keras.preprocessing.image import load_img, img_to_array, ImageDataGenerator
from sklearn.model_selection import train_test_split

# Set paths to your real and fake image folders
real_folder = r"D:\Deep_fake_detection\archive (1)\real_and_fake_face\training_real"
fake_folder = r"D:\Deep_fake_detection\archive (1)\real_and_fake_face\training_fake"

# Load and organize images
real_images = [os.path.join(real_folder, image) for image in os.listdir(real_folder)]
fake_images = [os.path.join(fake_folder, image) for image in os.listdir(fake_folder)]

# Combine real and fake images
all_images = real_images + fake_images
labels = [1] * len(real_images) + [0] * len(fake_images)

# Shuffle the images and labels
combined = list(zip(all_images, labels))
random.shuffle(combined)
all_images, labels = zip(*combined)

# Split the data into training, validation, and testing sets
train_images, test_images, train_labels, test_labels = train_test_split(all_images, labels, test_size=0.3, random_state=42)
val_images, test_images, val_labels, test_labels = train_test_split(test_images, test_labels, test_size=0.5, random_state=42)

# Define image size for resizing
image_size = (224, 224)

# Function to preprocess images
def preprocess_image(image_path):
    img = load_img(image_path, target_size=image_size)
    img = img_to_array(img) / 255.0  # Normalize pixel values to [0, 1]
    return img

# Preprocess training images
train_images = np.array([preprocess_image(image) for image in train_images])
train_labels = np.array(train_labels)

# Preprocess validation images
val_images = np.array([preprocess_image(image) for image in val_images])
val_labels = np.array(val_labels)

# Preprocess testing images
test_images = np.array([preprocess_image(image) for image in test_images])
test_labels = np.array(test_labels)

# Print dataset statistics
print("Train images:", train_images.shape)
print("Validation images:", val_images.shape)
print("Test images:", test_images.shape)
print("Train labels:", train_labels.shape)
print("Validation labels:", val_labels.shape)
print("Test labels:", test_labels.shape)

D:\Anaconda\lib\site-packages\scipy\__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.25.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


Train images: (1428, 224, 224, 3)
Validation images: (306, 224, 224, 3)
Test images: (307, 224, 224, 3)
Train labels: (1428,)
Validation labels: (306,)
Test labels: (307,)


In [17]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# Define the CNN model
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(train_images, train_labels, epochs=10, batch_size=32, validation_data=(val_images, val_labels))

# Evaluate the model on the test set
test_loss, test_acc = model.evaluate(test_images, test_labels)
print("Test accuracy:", test_acc)



Epoch 1/10
45/45 [==============================] - 60s 1s/step - loss: 0.7564 - accuracy: 0.5182 - val_loss: 0.6894 - val_accuracy: 0.5784
Epoch 2/10
45/45 [==============================] - 59s 1s/step - loss: 0.6865 - accuracy: 0.5791 - val_loss: 0.6922 - val_accuracy: 0.5261
Epoch 3/10
45/45 [==============================] - 46s 1s/step - loss: 0.6841 - accuracy: 0.5903 - val_loss: 0.6839 - val_accuracy: 0.5850
Epoch 4/10
45/45 [==============================] - 49s 1s/step - loss: 0.6769 - accuracy: 0.6057 - val_loss: 0.6846 - val_accuracy: 0.5621
Epoch 5/10
45/45 [==============================] - 48s 1s/step - loss: 0.6566 - accuracy: 0.6162 - val_loss: 0.6815 - val_accuracy: 0.5752
Epoch 6/10
45/45 [==============================] - 48s 1s/step - loss: 0.6285 - accuracy: 0.6513 - val_loss: 0.7289 - val_accuracy: 0.5458
Epoch 7/10
45/45 [==============================] - 49s 1s/step - loss: 0.5744 - accuracy: 0.7045 - val_loss: 0.7481 - val_accuracy: 0.5784
Epoch 8/10
45/45 [==

In [18]:
# Function to predict real or fake
def predict_real_or_fake(image_path):
    # Preprocess the image
    img = preprocess_image(image_path)
    img = np.expand_dims(img, axis=0)
    
    # Make a prediction
    prediction = model.predict(img)
    
    # Set a threshold (e.g., 0.5)
    threshold = 0.5
    
    # Classify as real or fake based on the threshold
    if prediction >= threshold:
        result = "Fake"
    else:
        result = "Real"
    
    return result

In [19]:
# Path to the uploaded image you want to predict
uploaded_image_path = r"D:\time pass\hbh.jpg"
# Predict whether the uploaded image is real or fake
prediction_result = predict_real_or_fake(uploaded_image_path)
print("Predicted Result:", prediction_result)


1/1 [==============================] - 0s 68ms/step
Predicted Result: Fake
